# Talking Data

In [54]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import sys
import time
import random
import tarfile
import zipfile
import StringIO
from IPython.display import display, Image

from scipy import ndimage
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import linear_model, decomposition, datasets, ensemble
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,precision_score, recall_score, f1_score, average_precision_score, accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

datadir=".\data\TalkingData\\"
# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

### some function def

In [71]:
def loadData(datadir,filename):
    # Load the wholesale customers dataset
    #data = pd.read_csv(filename)
    data = ''
    try:
        if zipfile.is_zipfile(datadir+filename):
            z = zipfile.ZipFile(datadir+filename)
            filename = z.open(filename[:-4])
        data = pd.read_csv(filename, parse_dates=True)   
        print ("Dataset has {} samples with {} features each.".format(*data.shape))
    except Exception as e:
        print ("Dataset could not be loaded. Is the dataset missing?")
        print(e)
    return data

def writeData(data,filename):
    # Load the wholesale customers dataset
    try:
        data.to_csv(filename, index=False)
    except Exception as e:
        print ("from",filenameDataset could not be written.")
        print(e)
    verify=[]
    try:
        with open(filename, 'r') as f:
            for line in f:
                verify.append(line)
        f.closed
        return verify[:5]
    except IOError:
        sys.std

## Import data

In [72]:
app_labels = loadData(datadir,"app_labels.csv.zip")
app_events = loadData(datadir,"app_events.csv.zip")
events = loadData(datadir,"events.csv.zip")
gender_age_test = loadData(datadir,"gender_age_test.csv.zip") 
gender_age_train = loadData(datadir,"gender_age_train.csv.zip") 
label_categories = loadData(datadir,"label_categories.csv.zip") 
phone_brand_device_model = loadData(datadir,"phone_brand_device_model.csv.zip") 

print (app_labels.head(5))

Dataset has 459943 samples with 2 features each.
Dataset has 32473067 samples with 4 features each.
Dataset has 3252950 samples with 5 features each.
Dataset has 112071 samples with 1 features each.
Dataset has 74645 samples with 4 features each.
Dataset has 930 samples with 2 features each.
Dataset has 187245 samples with 3 features each.
                app_id  label_id
0  7324884708820027918       251
1 -4494216993218550286       251
2  6058196446775239644       406
3  6058196446775239644       407
4  8694625920731541625       406
